<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# A Baseline Model for the AgrifieldNet India Competition

This notebook walks you through the steps to load the data and build a baseline model using Random Forests for `AgrifieldNet India Competition`.

## Radiant MLHub API


The Radiant MLHub API gives access to open Earth imagery training data for machine learning applications. You can learn more about the repository at the [Radiant MLHub site](https://mlhub.earth) and about the organization behind it at the [Radiant Earth Foundation site](https://radiant.earth).

Full documentation for the API is available at [docs.mlhub.earth](docs.mlhub.earth).

Each item in our collection is explained in json format compliant with [STAC](https://stacspec.org/) [label extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/label) definition.

## Dependencies

This notebook utilizes the [`radiant-mlhub` Python client](https://pypi.org/project/radiant-mlhub/) for interacting with the API. This notebook also utilizes the [`pandas` library](https://pandas.pydata.org/). If you are running this notebooks using Binder, then these dependencies have already been installed. If you are running this notebook locally, you will need to install these yourself.

See the official [`radiant-mlhub` docs](https://radiant-mlhub.readthedocs.io/) for more documentation of the full functionality of that library.

In [1]:
#import libraries

import os
import glob
import json
import getpass
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
from radiant_mlhub import Dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# DOWNLOAD DATA FROM MLHUB

In [2]:
#For simplicity we select 4 out 12 bands for the this baseline model

Full_bands = ['B01', 'B02', 'B03', 'B04','B05', 'B06', 'B07', 'B08','B8A', 'B09', 'B11', 'B12']

selected_bands = Full_bands[1:4]  + [Full_bands[-5]]  #'B02', 'B03', 'B04', 'B08'
selected_bands

['B02', 'B03', 'B04', 'B08']

In [3]:
#define dataset collection_id , assets and necessary paths to collections

main = 'ref_agrifieldnet_competition_v1'

assets = ['field_ids','raster_labels']

source_collection = f'{main}_source'
train_label_collection = f'{main}_labels_train'
test_label_collection = f'{main}_labels_test'

In [4]:
#Append your MLHUB_API_KEY after this cell is executed to download dataset

os.environ['MLHUB_API_KEY'] =  getpass.getpass(prompt="MLHub API Key: ")

dataset = Dataset.fetch(main)

my_filter = dict(
    ref_agrifieldnet_competition_v1_labels_train=assets,

    ref_agrifieldnet_competition_v1_labels_test=[assets[0]],

    ref_agrifieldnet_competition_v1_source=selected_bands 
)

dataset.download(collection_filter=my_filter)

MLHub API Key: ········


unarchive ref_agrifieldnet_competition_v1.tar.gz: 100%|██████████| 6186/6186 [00:01<00:00, 5793.46it/s]
filter by collection ids and asset keys: 231716it [00:00, 3408205.70it/s]         
download assets: 100%|██████████| 7905/7905 [16:28<00:00,  8.00it/s]


## Prepare Train data

- Load collection.json in labels_train collection's path and retrieve all unique folder ids into a list.
- Use unique folder ids to create a list of field.tif and raster_labels.tif paths for all tiles.
- Create competition_train_data dataframe for folder_ids and field_paths
- Create field_crop_pair dataframe using field_crop_extractor.
- Create train_data dataframe using the feature_extractor with argsss (competition_train_data, source_collection)
- Group processed dataset by fields and find the pixel average of across the entire field 
- Merge train_data dataframe and field_crop_pair dataframe on field_id
- Split train_df dataframe for model training and evaluation 

In [5]:
#load collection json and retrieve all unique folder ids 
#use all unique folder ids to create a list of field and label paths for all tiles

with open (f'{main}/{train_label_collection}/collection.json') as f:
    train_json = json.load(f)
    
train_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in train_json['links'][4:]]

train_field_paths = [f'{main}/{train_label_collection}/{train_label_collection}_{i}/field_ids.tif' for i in train_folder_ids]
train_label_paths = [f'{main}/{train_label_collection}/{train_label_collection}_{i}/raster_labels.tif' for i in train_folder_ids]

In [6]:
#create dataset for folder_ids and field_paths

competition_train_data = pd.DataFrame(train_folder_ids, columns=['unique_folder_id'])
competition_train_data['field_paths'] = train_field_paths
competition_train_data.head()

,unique_folder_id,field_paths
0,28852,ref_agrifieldnet_competition_v1/ref_agrifieldn...
1,d987c,ref_agrifieldnet_competition_v1/ref_agrifieldn...
2,ca1d4,ref_agrifieldnet_competition_v1/ref_agrifieldn...
3,2ec18,ref_agrifieldnet_competition_v1/ref_agrifieldn...
4,7575d,ref_agrifieldnet_competition_v1/ref_agrifieldn...


In [7]:
# PREPROCESS FIELDS AND CROPS IN TILES FOR TRAININIG

In [8]:
#Extract field_crop Pairs 

def field_crop_extractor(crop_field_files):
    field_crops = {}

    for label_field_file in tqdm(crop_field_files):
        with rasterio.open(f'{main}/{train_label_collection}/{train_label_collection}_{label_field_file}/field_ids.tif') as src:
            field_data = src.read()[0]
        with rasterio.open(f'{main}/{train_label_collection}/{train_label_collection}_{label_field_file}/raster_labels.tif') as src:
            crop_data = src.read()[0]
    
        for x in range(0, crop_data.shape[0]):
            for y in range(0, crop_data.shape[1]):
                field_id = str(field_data[x][y])
                field_crop = crop_data[x][y]

                if field_crops.get(field_id) is None:
                    field_crops[field_id] = []

                if field_crop not in field_crops[field_id]:
                    field_crops[field_id].append(field_crop)
    
    field_crop_map  =[[k, v[0]]  for k, v in field_crops.items() ]
    field_crop = pd.DataFrame(field_crop_map , columns=['field_id','crop_id'])

    return field_crop[field_crop['field_id']!='0']

In [9]:
field_crop_pair = field_crop_extractor(train_folder_ids)
field_crop_pair.head()

100%|██████████| 1165/1165 [01:36<00:00, 12.09it/s]


,field_id,crop_id
1,757,6
2,756,6
3,1372,5
4,1374,1
5,1986,4


In [10]:
field_crop_pair.shape

(5551, 2)

In [11]:
# Our goal is developing a pixel-based Random Forest model. So we will create an X variable
# such that, each row is a pixel and each column is one of the band observations mapped to its corresponding field. 


img_sh = 256
n_selected_bands= len(selected_bands)

n_obs = 1  #imagery per chip(no time series)

def feature_extractor(data_ ,   path ):
    '''
        data_: Dataframe with 'field_paths' and 'unique_folder_id' columns
        path: Path to source collections files

        returns: pixel dataframe with corresponding field_ids
        '''
    
    X = np.empty((0, n_selected_bands * n_obs))
    X_tile = np.empty((img_sh * img_sh, 0))
    X_arrays = []
        
    field_ids = np.empty((0, 1))

    for idx, tile_id in tqdm(enumerate(data_['unique_folder_id'])):
        
        field_src =   rasterio.open( data_['field_paths'].values[idx])
        field_array = field_src.read(1)
        field_ids = np.append(field_ids, field_array.flatten())
        
        
        bands_src = [rasterio.open(f'{main}/{path}/{path}_{tile_id}/{band}.tif') for band in selected_bands]
        bands_array = [np.expand_dims(band.read(1).flatten(), axis=1) for band in bands_src]
        
        X_tile = np.hstack(bands_array)

        X_arrays.append(X_tile)
        

    X = np.concatenate(X_arrays)
    
    data = pd.DataFrame(X, columns=selected_bands)

    data['field_id'] = field_ids

    return data[data['field_id']!=0]

In [12]:
train_data = feature_extractor(competition_train_data, source_collection)
train_data.head()

1165it [01:20, 14.42it/s]


,B02,B03,B04,B08,field_id
11031,39,38,38,61,757.0
11287,39,38,38,63,757.0
11288,39,38,37,65,757.0
11289,38,37,36,64,757.0
11543,39,38,38,64,757.0


In [13]:
# Each field has several pixels in| the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id

train_data_grouped = train_data.groupby(['field_id']).mean().reset_index()
train_data_grouped.field_id = [str(int(i)) for i in train_data_grouped.field_id.values]
train_data_grouped.head()

,field_id,B02,B03,B04,B08
0,1,42.444444,42.722222,48.000000,60.277778
1,2,42.000000,42.166667,47.666667,63.916667
2,3,42.687500,43.500000,49.187500,66.625000
3,4,42.466667,43.800000,47.733333,62.600000
4,5,43.238095,45.238095,49.285714,63.380952


In [14]:
# merge pixel dataframe to field_crop_pair dataframe

train_df = pd.merge(train_data_grouped, field_crop_pair , on='field_id' )
train_df.head()

,field_id,B02,B03,B04,B08,crop_id
0,1,42.444444,42.722222,48.000000,60.277778,1
1,2,42.000000,42.166667,47.666667,63.916667,1
2,3,42.687500,43.500000,49.187500,66.625000,1
3,4,42.466667,43.800000,47.733333,62.600000,2
4,5,43.238095,45.238095,49.285714,63.380952,2


In [15]:
train_df.tail(8)

,field_id,B02,B03,B04,B08,crop_id
5543,7322,36.500000,32.250000,26.750000,64.500000,4
5544,7323,36.000000,31.000000,24.090909,71.090909,4
5545,7324,36.533333,31.666667,25.333333,67.400000,4
5546,7326,39.000000,34.576923,30.653846,59.269231,9
5547,7327,37.851852,32.629630,26.555556,63.925926,9
5548,7328,40.100000,35.100000,29.650000,60.550000,9
5549,7331,40.130435,35.130435,30.000000,62.608696,9
5550,7332,39.653846,35.230769,29.423077,53.346154,36


In [16]:
train_df.shape

(5551, 6)

In [17]:
# split data for model training and evaluation 

X_train, X_test, y_train, y_test =  train_test_split(train_df.drop(['field_id', 'crop_id'], axis=1), train_df['crop_id'] , test_size=0.3, random_state=24)

In [18]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3885, 4), (1666, 4), (3885,), (1666,))

# MODEL TRAINING

In [19]:
# We ran a simple hyperparameter tuning for the number of trees, and concluded to use:
n_trees = 20

In [20]:
# Fitting the RF model
rf = RandomForestClassifier(n_estimators = n_trees, random_state = 0, n_jobs = 3)
rf.fit(X_train, y_train.astype(int))

RandomForestClassifier(n_estimators=20, n_jobs=3, random_state=0)

In [21]:
# trained classes

rf.classes_

array([ 1,  2,  3,  4,  5,  6,  8,  9, 13, 14, 15, 16, 36])

# MODEL EVALUATION

In [22]:
from sklearn.metrics import classification_report
y_pred_crop = rf.predict(X_test)

print(classification_report(y_test,y_pred_crop))

              precision    recall  f1-score   support

           1       0.64      0.81      0.71       616
           2       0.36      0.30      0.33       284
           3       0.00      0.00      0.00        30
           4       0.72      0.71      0.71       504
           5       0.00      0.00      0.00         6
           6       0.33      0.13      0.19        54
           8       0.00      0.00      0.00        14
           9       0.61      0.72      0.66        89
          13       0.33      0.08      0.12        13
          14       0.00      0.00      0.00         5
          15       0.00      0.00      0.00        12
          16       0.00      0.00      0.00         4
          36       0.25      0.09      0.13        35

    accuracy                           0.61      1666
   macro avg       0.25      0.22      0.22      1666
weighted avg       0.56      0.61      0.58      1666



/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Prepare Test data

- Load collection json and retrieve all unique folder ids 
- Use unique folder ids to create a list of field.tif paths for all tiles
- Create competition_test_data dataframe for folder_ids and field_paths
- Create test_data dataframe using the feature_extractor with argsss (competition_test_data, source_collection)
- Group processed dataset by fields and find the pixel average of across the entire field 

In [23]:
with open (f'{main}/{test_label_collection}/collection.json') as f:
    test_json = json.load(f)
    
test_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in test_json['links'][4:]]

test_field_paths = [f'{main}/{test_label_collection}/{test_label_collection}_{i}/field_ids.tif' for i in test_folder_ids]

In [24]:
competition_test_data = pd.DataFrame(test_folder_ids , columns=['unique_folder_id'])
competition_test_data['field_paths'] = test_field_paths
competition_test_data.head()

,unique_folder_id,field_paths
0,6199c,ref_agrifieldnet_competition_v1/ref_agrifieldn...
1,6c81d,ref_agrifieldnet_competition_v1/ref_agrifieldn...
2,1ebeb,ref_agrifieldnet_competition_v1/ref_agrifieldn...
3,586a2,ref_agrifieldnet_competition_v1/ref_agrifieldn...
4,65812,ref_agrifieldnet_competition_v1/ref_agrifieldn...


In [25]:
test_data = feature_extractor(competition_test_data,  source_collection)
test_data.head()

707it [00:34, 20.62it/s]


,B02,B03,B04,B08,field_id
35283,35,35,35,59,5407.0
35284,34,33,34,58,5407.0
35538,36,36,37,56,5407.0
35539,35,36,34,75,5407.0
35540,33,34,31,79,5407.0


In [26]:
# Each field has several pixels in| the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id

test_data_grouped = test_data.groupby(['field_id']).mean().reset_index()
test_data_grouped.field_id = [str(int(i)) for i in test_data_grouped.field_id.values]
test_data_grouped

,field_id,B02,B03,B04,B08
0,11,39.057971,38.420290,38.405797,64.405797
1,13,40.692308,41.307692,46.769231,65.461538
2,19,41.918367,42.530612,44.367347,66.265306
3,21,40.250000,40.750000,40.000000,69.583333
4,25,44.300000,47.200000,52.900000,71.200000
...,...,...,...,...,...
1525,7319,36.333333,31.222222,24.722222,67.833333
1526,7325,37.735294,32.205882,26.205882,66.970588
1527,7329,40.588235,35.823529,30.823529,59.352941
1528,7330,40.000000,34.750000,29.125000,63.500000


# Submit predictions with field_ids and class probabilites

- run predictions with trained model
- pass to multioutput predictions to csv file with field_id as index
- save output file as submission.csv

In [27]:
# extract crop_id-label dictionary

with open('ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_001c1/ref_agrifieldnet_competition_v1_labels_train_001c1.json') as ll:
    label_json = json.load(ll)

In [28]:
crop_dict = {asset.get('values')[0]:asset.get('summary') for asset in label_json['assets']['raster_labels']['file:values']}

In [29]:
crop_dict

{1: 'Wheat',
 2: 'Mustard',
 3: 'Lentil',
 4: 'No Crop',
 6: 'Sugarcane',
 8: 'Garlic',
 15: 'Potato',
 5: 'Green pea',
 16: 'Bersem',
 14: 'Coriander',
 13: 'Gram',
 9: 'Maize',
 36: 'Rice'}

In [30]:
def labeler(labeled):
    crop_label = np.array([crop_dict.get(f'{int(i)}') for i in labeled])
    return crop_label

In [31]:
predictions = rf.predict_proba(test_data_grouped.drop('field_id', axis=1 ))

crop_columns = [crop_dict.get(i) for i in rf.classes_]

test_df  = pd.DataFrame(columns= ['field_id'] + crop_columns)

test_df['field_id'] = test_data_grouped.field_id

test_df[crop_columns]= predictions 
test_df.to_csv('submission.csv', index=False)

In [32]:
test_df.head()

,field_id,Wheat,Mustard,Lentil,No Crop,Green pea,Sugarcane,Garlic,Maize,Gram,Coriander,Potato,Bersem,Rice
0,11,0.10,0.40,0.0,0.15,0.0,0.30,0.0,0.0,0.0,0.0,0.05,0.0,0.0
1,13,0.65,0.25,0.0,0.10,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,19,0.30,0.20,0.0,0.45,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,21,0.00,0.15,0.0,0.80,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,25,0.30,0.10,0.2,0.30,0.0,0.10,0.0,0.0,0.0,0.0,0.00,0.0,0.0
